# 作业
2（可选）将课程项目中提供的民法典（节选）文档构建为知识库，并构建对应的 RAG ，向其提出 2 个问题并查看生成结果。

### 依赖与环境

In [1]:
!uv add "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0 ipywidgets

Resolved 105 packages in 2ms
Audited 83 packages in 0.01ms


---

## 准备数据

In [1]:
# mfd_text_lines = []

from collections import OrderedDict

with open("mfd.md", "r") as file:
    file_text = file.read()
# 去重
mfd_text_lines = list(OrderedDict.fromkeys(file_text.split("\n\n")).keys())

mfd_text_lines

['## 中华人民共和国民法典',
 '### （二）物权编',
 '#### 第一章 一般规定',
 '**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。',
 '**第二百零五条** 本编调整因物的归属和利用产生的民事关系。',
 '**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。',
 '**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。',
 '**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。',
 '**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。',
 '**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。',
 '**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。',
 '**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。',
 '**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。',
 '**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。',
 '**第二百一十五条** 不动产登记簿由登记机构管理。\n不动产登记簿应当采用纸质形式或者电子形式。\n不动产登记簿采用电子形式的，应当备份。',
 '**第二百一十六

## 将数据加载到 Milvus

### 准备 Embedding 模型

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [124]:
# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus.model.hybrid import MGTEEmbeddingFunction

embedding_model = MGTEEmbeddingFunction(
    model_name="Alibaba-NLP/gte-multilingual-base",  # Defaults to `Alibaba-NLP/gte-multilingual-base`
)


# from pymilvus import model

# embedding_model = model.dense.Model2VecEmbeddingFunction(
#     model_source="minishlab/potion-base-32M",
# )


生成一个测试嵌入并打印其维度和前几个元素。

In [126]:
test_embedding = embedding_model.encode_queries(["民法典"])

print(embedding_model.dim)
print(test_embedding["dense"][0].tolist())
# print(test_embedding[0])

{'dense': 768, 'sparse': 250002}
[-0.08693218231201172, 0.06236881762742996, -0.04616500064730644, 0.0072378916665911674, -0.05039249733090401, -0.032914385199546814, -0.08415286242961884, 0.046225711703300476, 0.020842088386416435, -0.08260015398263931, -0.09806246310472488, 0.09295835345983505, -0.08044066280126572, 0.03360266238451004, -0.07924147695302963, 0.08348516374826431, 0.13265007734298706, 0.02271074801683426, -0.03359184414148331, 0.02751447632908821, 0.09757671505212784, 0.04217693954706192, -0.01931094191968441, -0.04620496183633804, -0.01227208785712719, 0.08469104766845703, 0.04060833528637886, -0.05087598040699959, -0.030460629612207413, 0.04460643231868744, 0.006751147098839283, 0.013259848579764366, 0.01642248034477234, 0.008251077495515347, 0.010336493141949177, -0.03480546548962593, -0.009205218404531479, 0.03867475315928459, -0.01166183315217495, -0.01259860023856163, 0.00035067438147962093, -0.009197498671710491, -0.005184337496757507, 0.033010564744472504, 0.00

### 创建 Collection

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

In [127]:
from pymilvus import MilvusClient
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 添加这行禁用tokenizers并行
milvus_client = MilvusClient(uri="./milvus.db")

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [129]:
mfd_collection_name = "mfd_rag_collection"
# 检查 collection 是否已存在，如果存在则删除它。
if milvus_client.has_collection(mfd_collection_name):
    milvus_client.drop_collection(mfd_collection_name)

milvus_client.create_collection(
    collection_name=mfd_collection_name,
    dimension=embedding_model.dim["dense"],
    metric_type="IP",  # 内积距离
    consistency_level="Strong",
    # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。
    # 更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [130]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(mfd_text_lines)["dense"]

for i, line in enumerate(tqdm(mfd_text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i].tolist(), "text": line})

print(data[0]["vector"][:5])
print(data[1]["vector"][:5])

Creating embeddings: 100%|██████████| 413/413 [00:00<00:00, 134324.41it/s]

[-0.08968749642372131, 0.09569638222455978, -0.03572220727801323, -0.031257353723049164, -0.06867773085832596]
[-0.09370429068803787, 0.05894465744495392, -0.0264251958578825, 0.05819319561123848, -0.06521742790937424]


In [131]:
milvus_client.insert(collection_name=mfd_collection_name, data=data)

{'insert_count': 413, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

## 构建 RAG

### 检索查询数据

指定问题
1. 第二百二十八条是什么？
2. "遗失物自发布招领公告之日起半年年内无人认领的，归国家所有" 这句是否正确？

In [132]:
question1 = "《民法典》第二百二十八条是什么？"
question2 = "`遗失物自发布招领公告之日起半年年内无人认领的，归国家所有` 这句是否正确？"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [ ]:
def search(questions: list[str]):
    encode_data = embedding_model.encode_queries(questions)
    data = [d.tolist() for d in encode_data["dense"]]

    return milvus_client.search(
        collection_name=mfd_collection_name,
        data=data,  # 将问题转换为嵌入向量
        limit=5,  # 返回前5个结果
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )


search_res = search([question1, question2])
search_res

data: [[{'id': 69, 'distance': 0.7869792580604553, 'entity': {'text': '**第二百六十八条** 法律对集体所有的财产有特别保护的，依照其规定。'}}, {'id': 89, 'distance': 0.7518926858901978, 'entity': {'text': '**第二百八十八条** 不动产相邻权利人应当按照有利生产、方便生活、团结互助、公平合理的原则，正确处理相邻关系。'}}, {'id': 131, 'distance': 0.7477129697799683, 'entity': {'text': '**第三百二十八条** 不动产权利人对自然流水的利用，应当在合理范围内，不得损害相邻不动产权利人的合法权益。'}}, {'id': 99, 'distance': 0.7468970417976379, 'entity': {'text': '**第二百九十八条** 不动产权利人因建造、修缮建筑物以及其他原因，应当对相邻不动产造成损害的，应当依法承担民事责任。'}}, {'id': 27, 'distance': 0.7412330508232117, 'entity': {'text': '**第二百二十八条** 动产物权设立和转让，合同生效时，所有权没有转移，标的物也未交付，标的物由出让人继续占有的，物权自合同生效时发生效力。\n因受让人取得占有的，所有权自交付时发生效力。'}}], [{'id': 34, 'distance': 0.8688369393348694, 'entity': {'text': '**第二百三十五条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。'}}, {'id': 153, 'distance': 0.865397036075592, 'entity': {'text': '**第三百四十九条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。'}}, {'id': 33, 'distance': 0.7151668071746826, 'entity': {'text': '**第二百三十四条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。'}}, {'id': 152, 'd

让我们看一下查询的搜索结果

In [137]:
import json

retrieved_lines_with_distances1 = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]

retrieved_lines_with_distances2 = [
    (res["entity"]["text"], res["distance"]) for res in search_res[1]
]
print(json.dumps(retrieved_lines_with_distances1, indent=4))
print(json.dumps(retrieved_lines_with_distances2, indent=4))

[
    [
        "**\u7b2c\u4e8c\u767e\u516d\u5341\u516b\u6761** \u6cd5\u5f8b\u5bf9\u96c6\u4f53\u6240\u6709\u7684\u8d22\u4ea7\u6709\u7279\u522b\u4fdd\u62a4\u7684\uff0c\u4f9d\u7167\u5176\u89c4\u5b9a\u3002",
        0.7869792580604553
    ],
    [
        "**\u7b2c\u4e8c\u767e\u516b\u5341\u516b\u6761** \u4e0d\u52a8\u4ea7\u76f8\u90bb\u6743\u5229\u4eba\u5e94\u5f53\u6309\u7167\u6709\u5229\u751f\u4ea7\u3001\u65b9\u4fbf\u751f\u6d3b\u3001\u56e2\u7ed3\u4e92\u52a9\u3001\u516c\u5e73\u5408\u7406\u7684\u539f\u5219\uff0c\u6b63\u786e\u5904\u7406\u76f8\u90bb\u5173\u7cfb\u3002",
        0.7518926858901978
    ],
    [
        "**\u7b2c\u4e09\u767e\u4e8c\u5341\u516b\u6761** \u4e0d\u52a8\u4ea7\u6743\u5229\u4eba\u5bf9\u81ea\u7136\u6d41\u6c34\u7684\u5229\u7528\uff0c\u5e94\u5f53\u5728\u5408\u7406\u8303\u56f4\u5185\uff0c\u4e0d\u5f97\u635f\u5bb3\u76f8\u90bb\u4e0d\u52a8\u4ea7\u6743\u5229\u4eba\u7684\u5408\u6cd5\u6743\u76ca\u3002",
        0.7477129697799683
    ],
    [
        "**\u7b2c\u4e8c\u767e\u4e5d\u5341

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [146]:
context1 = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances1]
)
context1

'**第二百六十八条** 法律对集体所有的财产有特别保护的，依照其规定。\n**第二百八十八条** 不动产相邻权利人应当按照有利生产、方便生活、团结互助、公平合理的原则，正确处理相邻关系。\n**第三百二十八条** 不动产权利人对自然流水的利用，应当在合理范围内，不得损害相邻不动产权利人的合法权益。\n**第二百九十八条** 不动产权利人因建造、修缮建筑物以及其他原因，应当对相邻不动产造成损害的，应当依法承担民事责任。\n**第二百二十八条** 动产物权设立和转让，合同生效时，所有权没有转移，标的物也未交付，标的物由出让人继续占有的，物权自合同生效时发生效力。\n因受让人取得占有的，所有权自交付时发生效力。'

In [147]:
context2 = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances2]
)
context2

'**第二百三十五条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。\n**第三百四十九条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。\n**第二百三十四条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n**第三百四十八条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n**第三百五十条** 权利人悬赏寻找遗失物的，领取遗失物时应当按照承诺向拾得人支付报酬。\n拾得人侵占遗失物的，无权请求保管费等费用，也无权请求权利人支付报酬。'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [148]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""


def user_prompt(question: str, context: str):
    return f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后用 <reference>和</reference> 标签括起来标注出处。
<context>
{context}
</context>
<question>
{question}
</question>
<reference>
</reference>
"""

In [149]:
user_prompt1 = user_prompt(question1, context1)
user_prompt1

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后用 <reference>和</reference> 标签括起来标注出处。\n<context>\n**第二百六十八条** 法律对集体所有的财产有特别保护的，依照其规定。\n**第二百八十八条** 不动产相邻权利人应当按照有利生产、方便生活、团结互助、公平合理的原则，正确处理相邻关系。\n**第三百二十八条** 不动产权利人对自然流水的利用，应当在合理范围内，不得损害相邻不动产权利人的合法权益。\n**第二百九十八条** 不动产权利人因建造、修缮建筑物以及其他原因，应当对相邻不动产造成损害的，应当依法承担民事责任。\n**第二百二十八条** 动产物权设立和转让，合同生效时，所有权没有转移，标的物也未交付，标的物由出让人继续占有的，物权自合同生效时发生效力。\n因受让人取得占有的，所有权自交付时发生效力。\n</context>\n<question>\n《民法典》第二百二十八条是什么？\n</question>\n<reference>\n</reference>\n'

In [150]:
user_prompt2 = user_prompt(question2, context2)
user_prompt2

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后用 <reference>和</reference> 标签括起来标注出处。\n<context>\n**第二百三十五条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。\n**第三百四十九条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。\n**第二百三十四条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n**第三百四十八条** 拾得遗失物，应当返还权利人。拾得人应当及时通知权利人领取，或者送交公安等有关部门。\n**第三百五十条** 权利人悬赏寻找遗失物的，领取遗失物时应当按照承诺向拾得人支付报酬。\n拾得人侵占遗失物的，无权请求保管费等费用，也无权请求权利人支付报酬。\n</context>\n<question>\n`遗失物自发布招领公告之日起半年年内无人认领的，归国家所有` 这句是否正确？\n</question>\n<reference>\n</reference>\n'

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [151]:
import os
from openai import OpenAI

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("Please set the DEEPSEEK_API_KEY environment variable.")

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [152]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt1},
    ],
)
print(f"question: {question1}")
print("\nanswer:\n")
print(response.choices[0].message.content)

question: 《民法典》第二百二十八条是什么？

answer:

《民法典》第二百二十八条是关于动产物权设立和转让的规定，具体内容如下：

**第二百二十八条** 动产物权设立和转让，合同生效时，所有权没有转移，标的物也未交付，标的物由出让人继续占有的，物权自合同生效时发生效力。因受让人取得占有的，所有权自交付时发生效力。

<reference>
《中华人民共和国民法典》第二百二十八条
</reference>


In [153]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt2},
    ],
)
print(f"question: {question2}")
print("\nanswer:\n")
print(response.choices[0].message.content)

question: `遗失物自发布招领公告之日起半年年内无人认领的，归国家所有` 这句是否正确？

answer:

<reference>
**第二百三十五条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。  
**第三百四十九条** 遗失物自发布招领公告之日起一年内无人认领的，归国家所有。
</reference>

根据提供的上下文，原句中的“半年”是错误的。正确的表述应为“遗失物自发布招领公告之日起**一年内**无人认领的，归国家所有”。
